## creating a n-dimensional hypersphere 

encodes the RGB information of each pixel of an image.

In [2]:
from PIL import Image
from qiskit.quantum_info import Statevector

import numpy as np

coverting image to n-dimensional unit statevectors on an n-dimensional bloch sphere

In [5]:
image_path = "/Users/devaldeliwala/quantum_image_encryption/images/el_primo.jpg"
n = 6

im = Image.open(image_path, 'r')
image = im.convert("RGB")

n_pixels = np.array(image).shape[0]**2
pixel_values = np.array(list(image.getdata()))

def rgb_to_decimal(r, g, b):
    return (r << 16) + (g << 8) + b

def decimal_to_rgb(decimal):
    r = (decimal >> 16) & 0xFF
    g = (decimal >> 8) & 0xFF
    b = decimal & 0xFF
    return r, g, b

int_values = []
for r,g,b in pixel_values: 
    int_values.append(rgb_to_decimal(r, g, b))
int_values = np.array(int_values)

recovered_rgb_values = []
for i in int_values: 
    recovered_rgb_values.append(decimal_to_rgb(i))
recovered_rgb_values = np.array(recovered_rgb_values)

# checking for reversibility
if(np.allclose(pixel_values, recovered_rgb_values)): 
   print("rgb-int reversible conversion success")

rgb-int reversible conversion success


In [81]:
groups = []

i = 0 
while i < n_pixels: 
    groups.append(int_values[i:i + 2 ** n])
    i += 2**n

def to_unit_vector(arr):
    magnitude = np.linalg.norm(arr)
    if magnitude == 0:
        return arr, magnitude
    unit_vector = arr / magnitude
    return unit_vector, magnitude

def from_unit_vector(unit_vector, magnitude):
    return unit_vector * magnitude

unit_vector_groups = []
magnitudes = []

# unit_vector_groups stores 2^n-dimensional unit vectors 
for i in groups: 
    unit_vector, magnitude = to_unit_vector(i)
    magnitudes.append(magnitude)
    unit_vector_groups.append(unit_vector)

In [84]:
# converting each of the 2^n dimensional unit vectors reversibly into 
# n dimensional statevectors 

def unit_vector_to_statevector(unit_vector):
    dim = len(unit_vector) // 2
    real_parts = unit_vector[:dim]
    imag_parts = unit_vector[dim:]
    statevector = real_parts + 1j * imag_parts
    statevector /= np.linalg.norm(statevector)
    return statevector

statevectors = []
for i in unit_vector_groups: 
    statevectors.append(unit_vector_to_statevector(i))

statevectors = [Statevector(i) for i in statevectors]

for i in statevectors:
    display(i.draw('latex'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

converting the n-dimensional statevectors back into the RGB values that encode the image

In [142]:
def statevector_to_unit_vector(statevector):
    real_parts = np.real(statevector)
    imag_parts = np.imag(statevector)
    combined = np.concatenate([real_parts, imag_parts])
    unit_vector = combined / np.linalg.norm(combined)
    return unit_vector

recovered_unit_vectors = []

for i in statevectors:
    recovered_unit_vectors.append(statevector_to_unit_vector(i.data))


int_vectors = []
for i in range(len(recovered_unit_vectors)): 
    int_vectors.append(recovered_unit_vectors[i] * magnitudes[i])

recovered_int_values = []
for i in int_vectors: 
    for j in i: 
        recovered_int_values.append(j)
    
recovered_int_values = np.array(recovered_int_values)

recovered_pixel_values = []
for i in recovered_int_values: 
    recovered_pixel_values.append(decimal_to_rgb(int(i)))
recovered_pixel_values = np.array(recovered_rgb_values)


In [143]:
width, height = image.size

reshaped_pixel_values = recovered_pixel_values.reshape((height, width ,3))
reconstructed_image = Image.fromarray(reshaped_pixel_values.astype(np.uint8), 'RGB')
reconstructed_image.show()